# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

0.语言模型（1-gram）

In [2]:
chinese_dataset = 'article_9k.txt'

In [3]:
CHINESE_CHARATERS = open(chinese_dataset, encoding ='utf-8').read()

In [4]:
import pinyin

In [7]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [8]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [9]:
len(CHINESE_CHARATERS_COPYS)

129433034

In [10]:
import re

In [11]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [12]:
tokens(CHINESE_CHARATERS_COPYS[:10])

['ci', 'wai', 'zi']

In [18]:
from collections import Counter, defaultdict

In [19]:
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [20]:
PINYIN_COUNT.most_common(10)

[('shi', 860634),
 ('de', 809887),
 ('n', 691114),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441)]

In [63]:
def price(string):
    if string  not in PINYIN_COUNT:
        return 0.0
    else:
        return PINYIN_COUNT[string]/sum(PINYIN_COUNT[i] for i in PINYIN_COUNT )

In [64]:
price('ci')

0.0030673301788907686

1.切词

In [125]:
def pinyin_split(word):
    if word in memory:
        return memory[word]
    else:
        # 使用 pinyin_split(word[:i]) * pinyin_split(word[i:]概率连乘获得拆分后拼音的概率值
        max_prob, pos = max( [(price(word), 0)] 
                            + [( pinyin_split(word[i:]) * pinyin_split(word[:i]), i) for i in range(1, len(word))])
        # 按pos拆分概率最大，存储到字典solution
        solution[word] = (word[:pos], word[pos:])
        
        memory[word] = max_prob
        
        return max_prob

In [126]:
pinyin_split('shangdaxue')

1.3180791971961196e-07

In [127]:
solution

defaultdict(None,
            {'a': ('', 'a'),
             'u': ('', 'u'),
             'ua': ('u', 'a'),
             'h': ('', 'h'),
             'hu': ('', 'hu'),
             'hua': ('', 'hua'),
             'g': ('', 'g'),
             'gh': ('g', 'h'),
             'ghu': ('g', 'hu'),
             'ghua': ('g', 'hua'),
             'n': ('', 'n'),
             'ng': ('n', 'g'),
             'ngh': ('ng', 'h'),
             'nghu': ('ng', 'hu'),
             'nghua': ('n', 'ghua'),
             'i': ('', 'i'),
             'in': ('i', 'n'),
             'ing': ('in', 'g'),
             'ingh': ('in', 'gh'),
             'inghu': ('ing', 'hu'),
             'inghua': ('in', 'ghua'),
             'q': ('', 'q'),
             'qi': ('', 'qi'),
             'qin': ('', 'qin'),
             'qing': ('', 'qing'),
             'qingh': ('qing', 'h'),
             'qinghu': ('qing', 'hu'),
             'qinghua': ('qing', 'hua'),
             'gq': ('g', 'q'),
             'gqi': ('g', '

In [128]:
def parse_solution(word):
    left_split, right_split = solution[word]
    
    if left_split == '': return [right_split]
    return parse_solution(left_split) + parse_solution(right_split)

In [129]:
p = parse_solution('shangdaxue')
p

['shang', 'da', 'xue']

2.词修正，对应不同的概率

In [130]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [131]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [132]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [135]:
def correct_sequence_pinyin(p):
    return ' '.join(map(correct, p))

In [137]:
correct_sequence_pinyin((p))

'shang da xue'